# Normalización, renombrado y troceado de audio

In [11]:
import os
import numpy as np
import librosa
import soundfile as sf
from tqdm import tqdm

class AudioPreprocessor:
    def __init__(self, input_folder, output_folder, segment_duration=60):
        """
        Inicializa el preprocesador de audio
        
        :param input_folder: Carpeta con los archivos WAV originales
        :param output_folder: Carpeta donde se guardarán los archivos procesados
        :param segment_duration: Duración de cada segmento en segundos (default: 60)
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.segment_duration = segment_duration
        
        # Crear carpeta de salida si no existe
        os.makedirs(output_folder, exist_ok=True)

    def normalize_audio(self, audio):
        """Normaliza el audio a un rango de -1 a 1"""
        return librosa.util.normalize(audio)

    def segment_audio(self, audio, sr):
        """
        Divide el audio en segmentos de duración específica
        
        :param audio: Array de numpy con el audio
        :param sr: Frecuencia de muestreo
        :return: Lista de segmentos de audio
        """
        # Calcular el número de muestras por segmento
        segment_length = int(sr * self.segment_duration)
        
        # Calcular número de segmentos completos
        num_segments = len(audio) // segment_length
        
        segments = []
        for i in range(num_segments):
            start = i * segment_length
            end = start + segment_length
            segment = audio[start:end]
            segments.append(segment)
        
        # Manejar el último segmento si es necesario
        remaining = audio[num_segments * segment_length:]
        if len(remaining) > sr * 5:  # Solo incluir si es mayor a 5 segundos
            segments.append(remaining)
        
        return segments

    def process_file(self, filename):
        """
        Procesa un archivo de audio individual
        
        :param filename: Nombre del archivo a procesar
        """
        try:
            # Cargar el archivo
            file_path = os.path.join(self.input_folder, filename)
            print(f"\nProcesando: {filename}")
            
            # Cargar audio
            audio, sr = librosa.load(file_path, sr=None)
            print(f"Duración original: {len(audio)/sr:.2f} segundos")
            
            # Normalizar
            audio_normalized = self.normalize_audio(audio)
            
            # Segmentar
            segments = self.segment_audio(audio_normalized, sr)
            print(f"Número de segmentos: {len(segments)}")
            
            # Guardar segmentos
            base_name = os.path.splitext(filename)[0]
            for i, segment in enumerate(segments, 1):
                # Crear nombre de archivo
                new_filename = f"{base_name}_part{i:03d}.wav"
                output_path = os.path.join(self.output_folder, new_filename)
                
                # Guardar archivo
                sf.write(output_path, segment, sr, subtype='PCM_16')
                
            return len(segments)
            
        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")
            return 0

    def process_folder(self):
        """Procesa todos los archivos WAV en la carpeta de entrada"""
        try:
            # Obtener lista de archivos WAV
            wav_files = [f for f in os.listdir(self.input_folder) 
                        if f.lower().endswith('.wav')]
            
            if not wav_files:
                print("No se encontraron archivos WAV en la carpeta de entrada.")
                return
            
            print(f"Encontrados {len(wav_files)} archivos WAV")
            
            # Procesar cada archivo
            total_segments = 0
            for filename in tqdm(wav_files, desc="Procesando archivos"):
                segments = self.process_file(filename)
                total_segments += segments
            
            print(f"\nProcesamiento completado:")
            print(f"- Archivos procesados: {len(wav_files)}")
            print(f"- Segmentos totales generados: {total_segments}")
            
        except Exception as e:
            print(f"Error durante el procesamiento: {str(e)}")

# Uso del script
if __name__ == "__main__":
    input_folder = "/Volumes/Nexus/GRABACIONES DE CAMPO/2024/NOVIEMBRE/castro-dom-3-11-24/CRUDO/"
    output_folder = "/Volumes/Nexus/GRABACIONES DE CAMPO/2024/NOVIEMBRE/castro-dom-3-11-24/EDICIONES/"
    
    # Crear instancia del preprocesador
    processor = AudioPreprocessor(
        input_folder=input_folder,
        output_folder=output_folder,
        segment_duration=60  # Duración en segundos de cada segmento
    )
    
    # Procesar archivos
    processor.process_folder()

Encontrados 10 archivos WAV


Procesando archivos:   0%|          | 0/10 [00:00<?, ?it/s]


Procesando: ZOOM0013_LR.WAV
Duración original: 250.64 segundos
Número de segmentos: 5


Procesando archivos:  10%|█         | 1/10 [00:02<00:26,  2.93s/it]


Procesando: ZOOM0011_LR.WAV
Duración original: 665.56 segundos
Número de segmentos: 12


...


Procesando: ZOOM0018_LR.WAV
Duración original: 484.42 segundos
Número de segmentos: 8


Procesando archivos:  90%|█████████ | 9/10 [00:50<00:05,  5.95s/it]


Procesando: ZOOM0016_LR.WAV
Duración original: 455.24 segundos
Número de segmentos: 8


Procesando archivos: 100%|██████████| 10/10 [00:55<00:00,  5.55s/it]


Procesamiento completado:
- Archivos procesados: 10
- Segmentos totales generados: 84
